In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Estimación del variograma experimental

Este ejemplo utiliza el archivo UNCF_dataset.xlsx, que describe sondeos en los cuales se ha cortado una discordancia a una profunidad determinada.

- "X" >> Coordenada X de sondeo
- "Y" >> Coordenada Y de sondeo
- "ID" >> Identificador de sondeo
- "Elevation (ft)" >> Profundidad (en pies) de la discordancia (val. negativos)

In [ ]:
# Importamos módulos para el análisis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gstools as gs

# Posibles modelos de variograma
models = {
    "Gaussian": gs.Gaussian,
    "Exponential": gs.Exponential,
    "Stable": gs.Stable,
    "Rational": gs.Rational,
    "Circular": gs.Circular,
    "Spherical": gs.Spherical,
    "SuperSpherical": gs.SuperSpherical,
    "JBessel": gs.JBessel,
}

In [ ]:
# ESPEFICICAMOS ARCHIVO Y SEPARADOR DE COLUMNAS
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
FILE = "data/UNCF_dataset.xlsx"
HOJA = "Hoja1"
XCOL = "X"
YCOL = "Y"
VCOL = "Elevation (ft)"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

data = pd.read_excel(FILE, HOJA, engine = "openpyxl")
data.describe()

In [ ]:
# Representamos variable
xi = data[XCOL]
yi = data[YCOL]
val = data[VCOL]*-1
plt.scatter(xi, yi, marker="s")

In [ ]:
# ESTIMACIÓN DEL VARIOGRAMA EXPERIMENTAL
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
bins = None
#bins = np.arange(400, 11000, 600)
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


coords = [xi, yi]
bin_center, gamma = gs.vario_estimate(coords, val, bins)
fig, ax = plt.subplots()
ax.scatter(bin_center, gamma)
ax.set_ylabel("$\gamma$")
res = ax.set_xlabel("Distance")

In [ ]:
# AJUSTE MANUAL DE UN MODELO DE VARIOGRAMA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
C = 5500
a = 4000
Co = False
MODELO = "Gaussian"
XMAX = 11000
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

fit_model = models[MODELO](dim=2, var=C, len_scale=a, nugget=Co)
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()
res = fit_model.plot(x_max=XMAX, ax=ax)
ax.set_ylabel("$\gamma$")
res = ax.set_xlabel("Distance")


In [ ]:
# AJUSTE AUTOMÁTICO DEL MODELO DE VARIOGRAMA
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
PEPITA = False
MODELO = "Gaussian"
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

fit_model = models[MODELO](dim=2, nugget=PEPITA)
para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True)
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()
fit_model.plot(x_max=XMAX, ax=ax)
ax.set_ylabel("$\gamma$")
ax.set_xlabel("Distance")

print("PARÁMETROS DE AJUSTE")
print("="*20)
print("C: {0:.3f}".format(para["var"]))
print("a: {0:.3f}".format(para["len_scale"]))
print("Co: {0:.3f}".format(para["nugget"]))
print("R2: {0:.3f}".format(r2))

In [ ]:
# ESTIMACIÓN DEL MEJOR MODELO DE AJUSTE

# Representamos datos del variograma experimental
plt.scatter(bin_center, gamma, color="k", label="data")
ax = plt.gca()

# Ajustamos todos los modelos
scores = []
parameters = []
for model in models:
    try:
        fit_model = models[model](dim=2)
        para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True)
        var, len_scale, nugget = para["var"], para["len_scale"], para["nugget"]
        parameters.append([var, len_scale, nugget])
        fit_model.plot(x_max=XMAX, ax=ax)
        scores.append(r2)
    except:
        scores.append(0.000)
        parameters.append([0.0, 0.0, 0.0])


parameters = np.array(parameters)
scores = np.array(scores)
pos_scores = np.argsort(scores*-1)
modelos = np.array(list(models.keys()))[pos_scores]
parameters = parameters[pos_scores]
scores = scores[pos_scores]

for n in np.arange(scores.size):
    if scores[n] > 0:
        print("{0} {1} R2: {2:.3f} [C: {3:.2f}, a: {4:.2f}, Co: {5:.2f}]".format(n+1, modelos[n], scores[n], parameters[n,0], 
                                      parameters[n,1], parameters[n,2]))
    else:
        print("{0} {1} -- The model cannot be adjusted  --".format(n+1, modelos[n]))